#  Updates and GDPR using Delta Lake - Scala

In this notebook, we will review Delta Lake's end-to-end capabilities in Scala. You can also look at the original Quick Start guide if you are not familiar with [Delta Lake](https://github.com/delta-io/delta) [here](https://docs.delta.io/latest/quick-start.html). It provides code snippets that show how to read from and write to Delta Lake tables from interactive, batch, and streaming queries.

In this notebook, we will cover the following:

- Creating sample mock data containing customer orders
- Writing this data into storage in Delta Lake table format (or in short, Delta table)
- Querying the Delta table using functional and SQL
- The Curious Case of Forgotten Discount - Making corrections to data
- Enforcing GDPR on your data
- Oops, enforced it on the wrong customer! - Looking at the audit log to find mistakes in operations
- Rollback all the way!
- Closing the loop - 'defrag' your data

# Creating sample mock data containing customer orders

For this tutorial, we will setup a sample file containing customer orders with a simple schema: (order_id, order_date, customer_name, price).

In [20]:
spark.sql("DROP TABLE IF EXISTS input");
spark.sql("""
          CREATE TEMPORARY VIEW input 
          AS SELECT 1 order_id, '2019-11-01' order_date, 'Saveen' customer_name, 100 price
          UNION ALL SELECT 2, '2019-11-01', 'Terry', 50
          UNION ALL SELECT 3, '2019-11-01', 'Priyanka', 100
          UNION ALL SELECT 4, '2019-11-02', 'Steve', 10
          UNION ALL SELECT 5, '2019-11-03', 'Rahul', 10
          UNION ALL SELECT 6, '2019-11-03', 'Niharika', 75
          UNION ALL SELECT 7, '2019-11-03', 'Elva', 90
          UNION ALL SELECT 8, '2019-11-04', 'Andrew', 70
          UNION ALL SELECT 9, '2019-11-05', 'Michael', 20
          UNION ALL SELECT 10, '2019-11-05', 'Brigit', 25""")
var orders = spark.sql("SELECT * FROM input")
orders.show()
orders.printSchema()

res34: org.apache.spark.sql.DataFrame = []
res35: org.apache.spark.sql.DataFrame = []
orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]
+--------+----------+-------------+-----+
|order_id|order_date|customer_name|price|
+--------+----------+-------------+-----+
|       1|2019-11-01|       Saveen|  100|
|       2|2019-11-01|        Terry|   50|
|       3|2019-11-01|     Priyanka|  100|
|       4|2019-11-02|        Steve|   10|
|       5|2019-11-03|        Rahul|   10|
|       6|2019-11-03|     Niharika|   75|
|       7|2019-11-03|         Elva|   90|
|       8|2019-11-04|       Andrew|   70|
|       9|2019-11-05|      Michael|   20|
|      10|2019-11-05|       Brigit|   25|
+--------+----------+-------------+-----+

root
 |-- order_id: integer (nullable = false)
 |-- order_date: string (nullable = false)
 |-- customer_name: string (nullable = false)
 |-- price: integer (nullable = false)

# Writing this data into storage in Delta Lake table format (or in short, Delta table)

To create a Delta Lake table, you can write a DataFrame out in the **delta** format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta. These operations create a new Delta Lake table using the schema that was inferred from your DataFrame. 

If you already have existing data in Parquet format, you can do an "in-place" conversion to Delta Lake format. The code would look like following:

DeltaTable.convertToDelta(spark, $"parquet.`{path_to_data}`");

//Confirm that the converted data is now in the Delta format
DeltaTable.isDeltaTable(parquetPath)

In [21]:
val r = new scala.util.Random
val sessionId = r. nextInt(1000)
val path = s"/delta/delta-table-$sessionId";
path

r: scala.util.Random = scala.util.Random@2cc80ddc
sessionId: Int = 606
path: String = /delta/delta-table-606
res38: String = /delta/delta-table-606

In [22]:
// Here's how you'd do this in Parquet: 
// orders.repartition(1).write().format("parquet").save(path)

orders.repartition(1).write.format("delta").save(path)

# Querying the Delta table using functional and SQL


In [23]:
var ordersDataFrame = spark.read.format("delta").load(path)
ordersDataFrame.show()

ordersDataFrame: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]
+--------+----------+-------------+-----+
|order_id|order_date|customer_name|price|
+--------+----------+-------------+-----+
|       2|2019-11-01|        Terry|   50|
|       4|2019-11-02|        Steve|   10|
|       6|2019-11-03|     Niharika|   75|
|       8|2019-11-04|       Andrew|   70|
|       9|2019-11-05|      Michael|   20|
|       1|2019-11-01|       Saveen|  100|
|       3|2019-11-01|     Priyanka|  100|
|       5|2019-11-03|        Rahul|   10|
|       7|2019-11-03|         Elva|   90|
|      10|2019-11-05|       Brigit|   25|
+--------+----------+-------------+-----+

In [24]:
ordersDataFrame.createOrReplaceTempView("ordersDeltaTable")
spark.sql("SELECT * FROM ordersDeltaTable").show

+--------+----------+-------------+-----+
|order_id|order_date|customer_name|price|
+--------+----------+-------------+-----+
|       2|2019-11-01|        Terry|   50|
|       4|2019-11-02|        Steve|   10|
|       6|2019-11-03|     Niharika|   75|
|       8|2019-11-04|       Andrew|   70|
|       9|2019-11-05|      Michael|   20|
|       1|2019-11-01|       Saveen|  100|
|       3|2019-11-01|     Priyanka|  100|
|       5|2019-11-03|        Rahul|   10|
|       7|2019-11-03|         Elva|   90|
|      10|2019-11-05|       Brigit|   25|
+--------+----------+-------------+-----+

# Understanding Meta-data

In Delta Lake, meta-data is no different from data i.e., it is stored next to the data. Therefore, an interesting side-effect here is that you can peek into meta-data using regular Spark APIs. 

In [25]:
spark.read.text(s"$path/_delta_log/").collect.foreach(println);

[{"commitInfo":{"timestamp":1598847468922,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputBytes":"1293","numOutputRows":"10"}}}]
[{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}]
[{"metaData":{"id":"b8583e6a-ede5-46ac-98ef-de44ba53a466","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"order_id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"order_date\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"customer_name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"price\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1598847465653}}]
[{"add":{"path":"part-00000-6a8edcec-6641-47a3-b29d-88e03013c7e4-c000.snappy.parquet","partitionValues":{},"size":1293,"modificationTime":1598847468000,"dataChange

# The Curious Case of Forgotten Discount - Making corrections to data

Now that you are able to look at the orders table, you realize that you forgot to discount the orders that came in on November 1, 2019. Worry not! You can quickly make that correction.

In [26]:
import io.delta.tables._
import org.apache.spark.sql.functions._

var table = DeltaTable.forPath(path)

// Update every transaction that took place on November 1, 2019 and apply a discount of 10%
table.update(
  condition = expr("order_date == '2019-11-01'"),
  set = Map("price" -> expr("price - price*0.1")))

import io.delta.tables._
import org.apache.spark.sql.functions._
table: io.delta.tables.DeltaTable = io.delta.tables.DeltaTable@aa59a6d

In [27]:
table.toDF

res51: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [order_id: int, order_date: string ... 2 more fields]

When you now inspect the meta-data, what you will notice is that the original data is over-written. Well, not in a true sense but appropriate entries are added to Delta's transaction log so it can provide an "illusion" that the original data was deleted. We can verify this by re-inspecting the meta-data. You will see several entries indicating reference removal to the original data.

In [28]:
spark.read.text(s"$path/_delta_log/").collect.foreach(println)

[{"commitInfo":{"timestamp":1598847468922,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputBytes":"1293","numOutputRows":"10"}}}]
[{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}]
[{"metaData":{"id":"b8583e6a-ede5-46ac-98ef-de44ba53a466","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"order_id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"order_date\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"customer_name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"price\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1598847465653}}]
[{"add":{"path":"part-00000-6a8edcec-6641-47a3-b29d-88e03013c7e4-c000.snappy.parquet","partitionValues":{},"size":1293,"modificationTime":1598847468000,"dataChange

# Enforcing GDPR on your data

One of your customers wanted their data to be deleted. But wait, you are working with data stored on an immutable file system (e.g., HDFS, ADLS, WASB). How would you delete it? Using Delta Lake's Delete API.

Delta Lake provides programmatic APIs to conditionally update, delete, and merge (upsert) data into tables. For more information on these operations, see [Table Deletes, Updates, and Merges](https://docs.delta.io/latest/delta-update.html).

In [29]:
// Delete the appropriate customer
table.delete(condition = expr("customer_name == 'Saveen'"))
table.toDF.show

+--------+----------+-------------+-----+
|order_id|order_date|customer_name|price|
+--------+----------+-------------+-----+
|       2|2019-11-01|        Terry|   45|
|       4|2019-11-02|        Steve|   10|
|       6|2019-11-03|     Niharika|   75|
|       8|2019-11-04|       Andrew|   70|
|       9|2019-11-05|      Michael|   20|
|       3|2019-11-01|     Priyanka|   90|
|       5|2019-11-03|        Rahul|   10|
|       7|2019-11-03|         Elva|   90|
|      10|2019-11-05|       Brigit|   25|
+--------+----------+-------------+-----+

# Oops, enforced it on the wrong customer! - Looking at the audit/history log to find mistakes in operations

Delta's most powerful feature is the ability to allow looking into history i.e., the changes that were made to the underlying Delta Table. The cell below shows how simple it is to inspect the history.


In [30]:
table.history.drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").show(20, 1000, false)

+-------+-------------------+---------+-----------------------------------------------+-----------+-----------------------------------------------------------------------------------+
|version|          timestamp|operation|                            operationParameters|readVersion|                                                                   operationMetrics|
+-------+-------------------+---------+-----------------------------------------------+-----------+-----------------------------------------------------------------------------------+
|      2|2020-08-31 04:18:28|   DELETE|[predicate -> ["(`customer_name` = 'Saveen')"]]|          1|[numRemovedFiles -> 1, numDeletedRows -> 1, numAddedFiles -> 1, numCopiedRows -> 9]|
|      1|2020-08-31 04:18:14|   UPDATE|  [predicate -> (order_date#1779 = 2019-11-01)]|          0|[numRemovedFiles -> 1, numAddedFiles -> 1, numUpdatedRows -> 3, numCopiedRows -> 7]|
|      0|2020-08-31 04:17:49|    WRITE|     [mode -> ErrorIfExists, partitionBy 

# Rollback all the way using Time Travel!

You can query previous snapshots of your Delta Lake table by using a feature called Time Travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.

Once you run the cell below, you should see the first set of data, from before you overwrote it. Time Travel is an extremely powerful feature that takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see [Query an older snapshot of a table (time travel)](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

In [31]:
spark.read.format("delta").option("versionAsOf", "1").load(path).write.mode("overwrite").format("delta").save(path)

In [32]:
// Delete the correct customer - REMOVE
table.delete(condition = expr("customer_name == 'Rahul'"))
table.toDF.show

+--------+----------+-------------+-----+
|order_id|order_date|customer_name|price|
+--------+----------+-------------+-----+
|       2|2019-11-01|        Terry|   45|
|       4|2019-11-02|        Steve|   10|
|       6|2019-11-03|     Niharika|   75|
|       8|2019-11-04|       Andrew|   70|
|       9|2019-11-05|      Michael|   20|
|       1|2019-11-01|       Saveen|   90|
|       3|2019-11-01|     Priyanka|   90|
|       7|2019-11-03|         Elva|   90|
|      10|2019-11-05|       Brigit|   25|
+--------+----------+-------------+-----+

In [33]:
table.history.drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").show(20, 1000, false)

+-------+-------------------+---------+-----------------------------------------------+-----------+-----------------------------------------------------------------------------------+
|version|          timestamp|operation|                            operationParameters|readVersion|                                                                   operationMetrics|
+-------+-------------------+---------+-----------------------------------------------+-----------+-----------------------------------------------------------------------------------+
|      4|2020-08-31 04:19:02|   DELETE| [predicate -> ["(`customer_name` = 'Rahul')"]]|          3|[numRemovedFiles -> 1, numDeletedRows -> 1, numAddedFiles -> 1, numCopiedRows -> 9]|
|      3|2020-08-31 04:18:49|    WRITE|         [mode -> Overwrite, partitionBy -> []]|          2|                       [numFiles -> 1, numOutputBytes -> 1316, numOutputRows -> 10]|
|      2|2020-08-31 04:18:28|   DELETE|[predicate -> ["(`customer_name` = 'Savee

# Closing the loop - 'defrag' your data


In [34]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
table.vacuum(0.01)

// Alternate Syntax: spark.sql($"VACUUM delta.`{path}`").show

Deleted 4 files and directories in a total of 1 directories.
res63: org.apache.spark.sql.DataFrame = []